In [ ]:
! pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
! kaggle datasets download -d mikoajfish99/lions-or-cheetahs-image-classification
!unzip lions-or-cheetahs-image-classification.zip

In [ ]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split

# Путь к папке с изображениями
path = '/content/images'

# Список категорий животных
categories = ['Cheetahs', 'Lions']

# Размер изображений
img_size = 256

# Создаем список изображений и соответствующих им лейблов
data = []
labels = []
for category in categories:
    folder_path = os.path.join(path, category)
    img_names = os.listdir(folder_path)
    for img_name in img_names:
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (img_size, img_size))
        data.append(img)
        labels.append(categories.index(category))

# Конвертируем списки в numpy-массивы
data = np.array(data)
labels = np.array(labels)

# Разделяем на трейн и тест выборки
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)


In [ ]:
# import matplotlib.pyplot as plt
# from PIL import Image

# # загружаем изображение
# img = train_data[3]

# # создаем subplot и отображаем изображение
# fig, ax = plt.subplots(figsize=(8, 8))
# ax.imshow(img)

# # отображаем subplot
# plt.show()

In [ ]:
import tensorflow as tf
from keras.layers import Dense, Flatten
from keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50


# Загрузка предварительно обученной модели ResNet-50 без последнего слоя
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Указание формы выходных данных из слоя base_model.output
output_shape = base_model.output_shape[1:]

# Добавление нового слоя для классификации на два класса (львы и гепарды)
x = base_model.output
predictions = Dense(2, activation='softmax')(x)

# Создание новой модели на основе ResNet-50 и добавленного слоя
model = Model(inputs=base_model.input, outputs=predictions)

# Замораживание всех слоев, кроме последнего добавленного слоя
for layer in base_model.layers:
    layer.trainable = False


# Компиляция модели с функцией потерь, оптимизатором и метриками
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
from keras.utils import to_categorical

# Преобразование целевых меток в one-hot encoding
train_labels = to_categorical(train_labels, num_classes=2)
test_labels = to_categorical(test_labels, num_classes=2)

In [ ]:
# Обучение модели на вашем датасете
model.fit(train_data, train_labels,
          epochs=10,
          validation_data=(test_data, test_labels))

# Оценка производительности модели на тестовом датасете
test_loss, test_acc = model.evaluate(test_data, test_labels)
print('Test accuracy:', test_acc)

Epoch 1/10
5/5 [==============================] - 6s 625ms/step - loss: 0.6934 - accuracy: 0.6375 - val_loss: 0.6378 - val_accuracy: 0.7500
Epoch 2/10
5/5 [==============================] - 1s 147ms/step - loss: 0.3476 - accuracy: 0.8938 - val_loss: 0.5638 - val_accuracy: 0.7750
Epoch 3/10
5/5 [==============================] - 1s 145ms/step - loss: 0.2159 - accuracy: 0.9312 - val_loss: 0.4060 - val_accuracy: 0.7750
Epoch 4/10
5/5 [==============================] - 1s 145ms/step - loss: 0.1642 - accuracy: 0.9500 - val_loss: 0.3698 - val_accuracy: 0.8250
Epoch 5/10
5/5 [==============================] - 1s 147ms/step - loss: 0.1208 - accuracy: 0.9625 - val_loss: 0.4034 - val_accuracy: 0.8250
Epoch 6/10
5/5 [==============================] - 1s 146ms/step - loss: 0.1053 - accuracy: 0.9812 - val_loss: 0.3920 - val_accuracy: 0.8250
Epoch 7/10
5/5 [==============================] - 1s 147ms/step - loss: 0.0845 - accuracy: 0.9875 - val_loss: 0.3542 - val_accuracy: 0.8250
Epoch 8/10
5/5 [====

In [ ]:
# Предсказание метки класса
preds = model.predict(test_data)

2/2 [==============================] - 1s 140ms/step


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Определение меток классов
class_names = ['Cheetah', 'Lion']

# Визуализация первых 20 изображений с их предсказаниями
plt.figure(figsize=(15,15))
for i in range(20):
    plt.subplot(5,4,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_data[i], cmap=plt.cm.binary)
    predicted_label = np.argmax(preds[i])
    plt.xlabel("Predicted: " + class_names[predicted_label])
plt.show()
